In [1]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseData2.db')
c = conn.cursor()

In [2]:
%load_ext sql

In [3]:
%%sql
sqlite:///CourseData2.db

'Connected: @CourseData2.db'

In [4]:
%%sql
--Dropping tables for when we need to restart and clear CourseData database-- 
--Drop ERD tables--
DROP TABLE IF EXISTS SECTION;
DROP TABLE IF EXISTS CATALOG;
DROP TABLE IF EXISTS INSTRUCTOR;
DROP TABLE IF EXISTS COURSE_MEETING;
DROP TABLE IF EXISTS PROGRAM;
DROP TABLE IF EXISTS TIME;

--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Create Tables (Import and ERD) ##

In [5]:
%%sql
--Creating import_catalog table-- 

CREATE TABLE 'import_course_catalog' (
    program_code TEXT NOT NULL,
    program_name VARCHAR NOT NULL,
    catalog_id VARCHAR NOT NULL,
    course_title VARCHAR NOT NULL, 
    credits VARCHAR NOT NULL,
    prereqs VARCHAR, 
    coreqs VARCHAR,
    fees VARCHAR,
    attributes VARCHAR,
    description VARCHAR 
);

 * sqlite:///CourseData2.db
Done.


[]

In [6]:
%%sql
--Creating import_course_meeting table-- 

CREATE TABLE 'import_course_meeting' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    location VARCHAR NOT NULL,
    day TEXT NOT NULL,
    start VARCHAR NOT NULL,
    end VARCHAR NOT NULL 
);

 * sqlite:///CourseData2.db
Done.


[]

In [7]:
%%sql
--Creating import_course table-- 

CREATE TABLE 'import_course' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    catalog_id VARCHAR NOT NULL,
    section VARCHAR NOT NULL,
    credits VARCHAR,
    title VARCHAR, 
    meetings VARCHAR,
    timecodes VARCHAR, 
    primary_instructor TEXT,
    cap INTEGER, 
    act INTEGER, 
    rem INTEGER 
);

 * sqlite:///CourseData2.db
Done.


[]

In [8]:
%%sql
-- Creating table INSTRUCTOR with surrogate primary key-- 

CREATE TABLE 'INSTRUCTOR' (
    InID INTEGER NOT NULL PRIMARY KEY,
    Name TEXT NOT NULL
);


--Creating table PROGRAM with PID as the primary key-- 
CREATE TABLE 'PROGRAM'(
    PID INTEGER PRIMARY KEY,
    PName TEXT NOT NULL,
    PCode TEXT NOT NULL
);

--Creating table CATALOG with CatID as the primary key-- 

CREATE TABLE 'CATALOG'(
    CatClass INTEGER PRIMARY KEY,
    CatID VARCHAR,
    Title VARCHAR NOT NULL,
    Description TEXT,
    Credits TEXT NOT NULL,
    PID INTEGER NOT NULL,
    PREREQS TEXT,
    COREQS TEXT,
    FEES INTEGER,
    Attribute TEXT,
    FOREIGN KEY (PID) REFERENCES PROGRAM(PID)
    
);

--Creating table SECTION with surrogate primary key-- 

CREATE TABLE 'SECTION'(
    SID INTEGER NOT NULL PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Section VARCHAR NOT NULL,
    Cap INTEGER,
    Act INTEGER,
    Rem INTEGER,
    CatID VARCHAR,
    InID INTEGER,
    FOREIGN KEY (InID) REFERENCES INSTRUCTOR(InID)
);

--Creating table TIME with TID as the Primary Key--

CREATE TABLE 'TIME'(
    TID INTEGER PRIMARY KEY,
    Start TEXT NOT NULL,
    End TEXT NOT NULL, 
    Day TEXT NOT NULL
);

--Creating table COURSE_MEETING with CatalogClassifier as the Primary Key-- 

CREATE TABLE 'COURSE_MEETING' (
    MID INTEGER NOT NULL PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Location VARCHAR NOT NULL,
    TID INTEGER NOT NULL,
    FOREIGN KEY (TID) REFERENCES TIME(TID),
    FOREIGN KEY (CRN) REFERENCES SECTION(CRN),
    FOREIGN KEY (Term) REFERENCES SECTION(Term)
);

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Populate the Import Tables with the Data from the csv files ##

**Creating dataframes for the import course**

In [9]:
## populating import_course table
fall = ['SourceData/Fall2014/courses.csv','SourceData/Fall2015/courses.csv', 'SourceData/Fall2016/courses.csv',
        'SourceData/Fall2017/courses.csv','SourceData/Fall2018/courses.csv']
dff0 = pd.read_csv(fall[0])
dff1 = pd.read_csv(fall[1])
dff2 = pd.read_csv(fall[2])    
dff3 = pd.read_csv(fall[3])
dff4 = pd.read_csv(fall[4])

In [10]:
df_fall = pd.concat([dff0, dff1, dff2, dff3, dff4], axis=0)
df_fall.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
1040,Fall2014,72165,NS 0303,B,3.000,Basic Pathophysiology and Pharmacology,"[{'days': 'MR', 'times': '0930am-1045am', 'dat...",['MR 0930am-1045am 09/02-12/08 SON 203'],Rose Iannino-Renz,48,48,0
637,Fall2015,75093,FE 0001,P,0.000,First Year Experience,"[{'days': 'W', 'times': '0200pm-0315pm', 'date...",['W 0200pm-0315pm 09/01-12/07 CNS 208'],Kamala Kiem,27,22,5
1125,Fall2016,77771,NS 0646,01,4.000,"Practicum in Care of Women, Children, and Fami...","[{'days': 'W', 'times': '0740pm-0840pm', 'date...",['W 0740pm-0840pm 09/06-12/19 BNW 340'],Jaclyn Elizabeth Conelius,15,10,5
9,Fall2016,77935,AC 0011,10,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1100am-1215pm', 'dat...",['TF 1100am-1215pm 09/06-12/12 DSB 110B'],Jo Ann Drusbosky,29,27,2
161,Fall2018,74512,BI 0170P,12,0.000,General Biology I PLG,"[{'days': 'R', 'times': '0600pm-0650pm', 'date...",['R 0600pm-0650pm 09/04-12/10 BNW 319'],TBA,12,8,4


In [11]:
df_fall.describe()

,crn,cap,act,rem
count,7486.000000,7486.00000,7486.000000,7486.000000
mean,74523.769169,18.39487,16.310446,2.219076
std,2512.545835,14.41373,9.863494,17.175757
min,70001.000000,0.00000,-1.000000,-75.000000
25%,72213.000000,12.00000,8.000000,0.000000
50%,74848.500000,19.00000,16.000000,1.000000
75%,76698.750000,25.00000,24.000000,5.000000
max,79215.000000,500.00000,90.000000,999.000000


In [12]:
spring = ['SourceData/Spring2015/courses.csv','SourceData/Spring2016/courses.csv', 'SourceData/Spring2017/courses.csv',
        'SourceData/Spring2018/courses.csv','SourceData/Spring2019/courses.csv', 'SourceData/SpringBreak2017/courses.csv']
dfs0 = pd.read_csv(spring[0])
dfs1 = pd.read_csv(spring[1])
dfs2 = pd.read_csv(spring[2])    
dfs3 = pd.read_csv(spring[3])
dfs4 = pd.read_csv(spring[4])
dfs5 = pd.read_csv(spring[5])

In [13]:
df_spring = pd.concat([dfs0,dfs1,dfs2,dfs3,dfs4,dfs5], axis=0)
df_spring.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
543,Spring2016,37984,FTM 0011,A,3.000,"Lights, Camera, Audio","[{'days': 'R', 'times': '1100am-0130pm', 'date...",['R 1100am-0130pm 01/19-05/03 XVR 5'],Patrick Brooks,23,19,4
918,Spring2017,31035,MA 0391,01,3.000,Honors Seminar II,"[{'days': 'TF', 'times': '0930am-1020am', 'dat...","['TF 0930am-1020am 01/17-05/02 BNW GR22', 'W 0...",Janet Striuli,15,4,11
224,Spring2019,37299,CH 0086,01,3.000,Chemistry and Art,"[{'days': 'TR', 'times': '0500pm-0615pm', 'dat...",['TR 0500pm-0615pm 01/22-04/30 BNW 332'],Dorothy Sobczynski,20,19,1
833,Spring2015,31855,MA 0019,C,3.000,Introduction to Calculus,"[{'days': 'TWF', 'times': '1100am-1150am', 'da...",['TWF 1100am-1150am 01/20-04/30 BNW 253'],Joan Wyzkoski Weiss,35,22,13
1122,Spring2018,35307,PS 0116,02,3.000,General Physics II,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...","['TF 0800am-0915am 01/16-05/01 BNW 137', 'W 08...",Jonathan J. Stott,24,24,0


In [14]:
df_spring.describe()

,crn,cap,act,rem
count,7122.000000,7122.000000,7122.000000,7122.000000
mean,35212.001966,18.123982,15.658804,2.465178
std,2618.670948,10.876640,10.342885,6.497100
min,31001.000000,0.000000,0.000000,-47.000000
25%,32773.250000,11.000000,7.000000,0.000000
50%,34835.000000,19.000000,16.000000,1.000000
75%,37693.750000,25.000000,24.000000,4.000000
max,41702.000000,100.000000,90.000000,89.000000


In [15]:
summer = ['SourceData/Summer2015/courses.csv', 'SourceData/Summer2016/courses.csv',
        'SourceData/Summer2017/courses.csv','SourceData/Summer2018/courses.csv']
dfsm0 = pd.read_csv(summer[0])
dfsm1 = pd.read_csv(summer[1])
dfsm2 = pd.read_csv(summer[2])    
dfsm3 = pd.read_csv(summer[3])

In [16]:
df_summer = pd.concat([dfsm0,dfsm1,dfsm2,dfsm3], axis=0)
df_summer.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
181,Summer2016,56437,NS 0270L,04,0.000,Health Assessment Lab,[],[],Anka Roberto,12,0,12
239,Summer2015,55290,NSAN 0673,D01,3.000,Chemistry and Physics for Nurse Anesthetists,"[{'days': 'W', 'times': '1230pm-0330pm', 'date...",['W 1230pm-0330pm 05/18-08/21 BNW 334'],Nancy A. Moriber,15,15,0
160,Summer2017,57382,MA 0495B,01,3.000,Special Topics: Graph Theory,"[{'days': 'MW', 'times': '0600pm-0900pm', 'dat...",['MW 0600pm-0900pm 05/24-07/05 BNW 253'],Janet Striuli,15,8,7
235,Summer2016,56246,NS 0687,02,0.000-6.000,DNP Immersion,[],[],Nancy A. Moriber,30,13,17
44,Summer2016,56475,CO 0202,OL1,3.000,Small Group Communication,[],[],Michael P. Pagano,20,5,15


In [17]:
df_summer.describe()

,crn,cap,act,rem
count,1224.000000,1224.000000,1224.000000,1224.000000
mean,56704.305556,14.843954,10.913399,3.928922
std,1143.898206,10.771969,9.124774,6.881239
min,55001.000000,0.000000,-1.000000,-24.000000
25%,55382.750000,6.000000,5.000000,0.000000
50%,56517.500000,16.000000,9.000000,3.000000
75%,58008.500000,20.000000,15.000000,9.000000
max,58477.000000,60.000000,59.000000,24.000000


In [18]:
winter = ['SourceData/Winter2015/courses.csv', 'SourceData/Winter2016/courses.csv',
        'SourceData/Winter2017/courses.csv','SourceData/Winter2018/courses.csv']
dfw0 = pd.read_csv(winter[0])
dfw1 = pd.read_csv(winter[1])
dfw2 = pd.read_csv(winter[2])    
dfw3 = pd.read_csv(winter[3])

In [19]:
df_winter = pd.concat([dfw0,dfw1,dfw2,dfw3], axis=0)
df_winter.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
3,Winter2017,11701,BU 0211,01,3.000,Legal Environment of Business,"[{'days': 'MTWRF', 'times': '0830am-0430pm', '...",['MTWRF 0830am-0430pm 01/04-01/10 DSB 109'],Sharlene A. McEvoy,25,20,5
14,Winter2015,10918,ENW 0444,01,6.000,Fiction: Part I,[],[],Michael C. White,1,1,0
2,Winter2016,11604,AH 0172,OL1,3.000,History of Photography,[],[],Lauren Elise Cesiro,25,21,4
23,Winter2016,11611,PY 0121,OL1,3.000,Fundamentals of Social Psychology,[],[],Dorothea D. Braginsky,25,21,4
9,Winter2015,10898,EC 0011,01,3.000,Introduction to Microeconomics,[],[],William F. Vasquez Mazariegos,20,9,11


In [20]:
df_winter.describe()

,crn,cap,act,rem
count,105.000000,105.000000,105.000000,105.000000
mean,11524.266667,17.133333,12.342857,4.790476
std,355.382575,7.612069,6.584990,6.207463
min,10893.000000,0.000000,0.000000,-3.000000
25%,11603.000000,15.000000,9.000000,0.000000
50%,11633.000000,20.000000,12.000000,4.000000
75%,11802.000000,20.000000,18.000000,9.000000
max,11841.000000,30.000000,25.000000,27.000000


In [21]:
df_course = pd.concat([df_fall, df_spring, df_summer, df_winter], axis=0)
df_course.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
1019,Spring2016,37867,NS 0524,01,5.000,Quality Outcomes Management II,[],[],Sally O. Gerard,10,8,2
1127,Fall2016,77773,NS 0661,01,2.000,Mental Health Nursing of Groups and Families A...,"[{'days': 'R', 'times': '0500pm-0640pm', 'date...",['R 0500pm-0640pm 09/06-12/19 MCA 107'],Joyce Muriel Shea,15,14,1
495,Spring2017,32795,EN 0012,55,3.000,Texts and Contexts II: Writing About Literature,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 01/17-05/02 CNS 206'],Katherine V. Veilleux,19,19,0
86,Summer2018,58409,ED 0599,01,3.000,Professional Writing Seminar: Product of Learning,"[{'days': 'TR', 'times': '0400pm-0800pm', 'dat...","['TR 0400pm-0800pm 05/08-05/17 CNS 5', 'TR 040...",Bernadine Krawczyk,4,2,2
437,Spring2017,32766,EG 0260,01,3.000,Robots,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 01/17-05/02 BNW 138'],Ryan A. Munden,15,13,2


In [22]:
df_course.describe()

,crn,cap,act,rem
count,15937.000000,15937.000000,15937.000000,15937.000000
mean,55172.299178,17.992784,15.578591,2.477317
std,19308.879968,12.671906,10.107690,12.709781
min,10893.000000,0.000000,-1.000000,-75.000000
25%,35426.000000,10.000000,7.000000,0.000000
50%,57145.000000,19.000000,15.000000,1.000000
75%,74541.000000,25.000000,23.000000,5.000000
max,79215.000000,500.000000,90.000000,999.000000


In [23]:
df_course.to_sql('import_course', conn, if_exists='append', index=False)

In [24]:
%%sql
SELECT *
FROM import_course
LIMIT 3;

 * sqlite:///CourseData2.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31


**Creating dataframes for the import course_meeting**

In [25]:
cm_fall = ['SourceData/Fall2014/course_meetings.csv', 'SourceData/Fall2015/course_meetings.csv',
           'SourceData/Fall2016/course_meetings.csv', 'SourceData/Fall2017/course_meetings.csv',
          'SourceData/Fall2018/course_meetings.csv']
dfcmf1 = pd.read_csv(cm_fall[0])
dfcmf2  = pd.read_csv(cm_fall[1])
dfcmf3  = pd.read_csv(cm_fall[2])
dfcmf4  = pd.read_csv(cm_fall[3])
dfcmf5  = pd.read_csv(cm_fall[4])

In [26]:
df_cmfall = pd.concat([dfcmf1, dfcmf2, dfcmf3, dfcmf4, dfcmf5], axis=0)
df_cmfall.sample(5)

,term,crn,location,day,start,end
19094,Fall2016,77620,BNW GR22,R,2016-12-08T12:30:00,2016-12-08T13:45:00
22154,Fall2016,77628,STM,F,2016-11-18T07:00:00,2016-11-18T13:00:00
19103,Fall2016,77620,BNW GR22,M,2016-10-03T12:00:00,2016-10-03T12:50:00
18803,Fall2017,72127,BNW 138,M,2017-12-11T10:00:00,2017-12-11T10:50:00
22969,Fall2018,74946,NHS 404,M,2018-11-05T14:00:00,2018-11-05T15:15:00


In [27]:
cm_spring = ['SourceData/Spring2015/course_meetings.csv',
           'SourceData/Spring2016/course_meetings.csv', 'SourceData/Spring2017/course_meetings.csv',
          'SourceData/Spring2018/course_meetings.csv', 'SourceData/Spring2019/course_meetings.csv']
dfcms1 = pd.read_csv(cm_spring[0])
dfcms2  = pd.read_csv(cm_spring[1])
dfcms3  = pd.read_csv(cm_spring[2])
dfcms4  = pd.read_csv(cm_spring[3])
dfcms5  = pd.read_csv(cm_spring[4])


In [28]:
df_cmspring = pd.concat([dfcms1, dfcms2, dfcms3, dfcms4, dfcms5], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
4198,Spring2015,33466,BNW 256,W,2015-02-11T09:00:00,2015-02-11T09:50:00
9066,Spring2016,37130,MCA 107,T,2016-04-12T14:00:00,2016-04-12T15:15:00
9675,Spring2019,37276,DMH 149,W,2019-03-06T08:00:00,2019-03-06T10:30:00
5502,Spring2016,37305,CNS 305,T,2016-04-12T09:30:00,2016-04-12T10:45:00
22476,Spring2017,31585,CNS 203,F,2017-01-27T09:30:00,2017-01-27T10:45:00


In [29]:
cm_springbreak = ['SourceData/SpringBreak2017/course_meetings.csv']

dfcmsb1 = pd.read_csv(cm_springbreak[0])


In [30]:
cm_summer = ['SourceData/Summer2015/course_meetings.csv',
           'SourceData/Summer2016/course_meetings.csv', 'SourceData/Summer2017/course_meetings.csv',
          'SourceData/Summer2018/course_meetings.csv']
dfcmsu1 = pd.read_csv(cm_summer[0])
dfcmsu2  = pd.read_csv(cm_summer[1])
dfcmsu3  = pd.read_csv(cm_summer[2])
dfcmsu4  = pd.read_csv(cm_summer[3])

In [31]:
df_cmsummer = pd.concat([dfcmsu1, dfcmsu2, dfcmsu3, dfcmsu4], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
29234,Spring2015,33935,PTH MAIN,F,2015-03-27T12:30:00,2015-03-27T13:45:00
7858,Spring2018,35249,BNW 133,W,2018-04-18T18:30:00,2018-04-18T21:00:00
19311,Spring2016,38945,BNW 138,F,2016-03-18T11:00:00,2016-03-18T11:50:00
3844,Spring2015,32799,BNW 256,W,2015-04-01T10:00:00,2015-04-01T10:50:00
21861,Spring2018,35194,CNS 305,T,2018-01-23T18:30:00,2018-01-23T19:45:00


In [32]:
cm_winter = ['SourceData/Winter2015/course_meetings.csv',
           'SourceData/Winter2016/course_meetings.csv', 'SourceData/Winter2017/course_meetings.csv',
          'SourceData/Winter2018/course_meetings.csv']
dfcmw1 = pd.read_csv(cm_winter[0])
dfcmw2  = pd.read_csv(cm_winter[1])
dfcmw3  = pd.read_csv(cm_winter[2])
dfcmw4  = pd.read_csv(cm_winter[3])

In [33]:
df_cmwinter = pd.concat([dfcmw1, dfcmw2, dfcmw3, dfcmw4], axis=0)
df_cmwinter.sample(5)

,term,crn,location,day,start,end
60,Winter2015,10909,DSB 111,W,2015-01-07T08:30:00,2015-01-07T16:30:00
52,Winter2016,11617,DSB 109,M,2016-01-04T18:00:00,2016-01-04T21:30:00
27,Winter2018,11834,DSB 106,F,2018-01-05T18:30:00,2018-01-05T21:30:00
9,Winter2016,11614,DSB 109,M,2016-01-11T08:30:00,2016-01-11T16:30:00
15,Winter2015,10913,DSB 115,M,2015-01-12T08:30:00,2015-01-12T16:30:00


In [34]:
df_coursemeeting = pd.concat([df_cmfall,df_cmspring,dfcmsb1,df_cmsummer,df_cmwinter],axis=0)

In [35]:
df_coursemeeting.to_sql('import_course_meeting', conn, if_exists='append', index=False)

In [36]:
%%sql
SELECT *
FROM import_course_meeting
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


term,crn,location,day,start,end
Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
Fall2014,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
Fall2014,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
Fall2014,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00


**Creating the dataframes for import_course_catalog**

In [37]:
cc2017_2018 = ['SourceData/Catalogs/CourseCatalog2017_2018.csv']
dfcc17_18 = pd.read_csv(cc2017_2018[0])
dfcc17_18.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1577,CR,Computer Engineering,CR 0346,Computer System Architecture,3 Credits,CR 0245.,NaN,NaN,NaN,This course introduces the machine language an...
413,SE,Special Education,SE 0400,Augmentative Alternative Communications and As...,3 Credits,NaN,NaN,NaN,NaN,This course provides prospective teachers with...
586,PY,Psychology,PY 0121,Fundamentals of Social Psychology,3 Credits,NaN,NaN,NaN,NaN,This course surveys the major areas of concern...
678,RS,Religious Studies,RS 0213,Jews and Judaism in America,3 Credits,RS 0101.,NaN,NaN,"ASRS American Studies: Religion, ASUP American...",What has it meant and what does it mean today ...
471,SE,Special Education,SE 0417,Introduction to Children and Youth with Social...,3 Credits,NaN,NaN,NaN,NaN,This core course introduces candidates to emot...


In [38]:
cc2018_2019 = ['SourceData/Catalogs/CourseCatalog2018_2019.csv']
dfcc18_19 = pd.read_csv(cc2018_2019[0])
dfcc18_19.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1804,ECE,Electrical and Computer Engineering,ECE 0591,Capstone Professional Project I,3 Credits,Completion of 9 credits in the MS ECE program.,NaN,NaN,NaN,"In these two semester capstone courses, studen..."
341,TA,Theatre,TA 0230,Acting II,3 Credits,TA 0030.,NaN,NaN,VPC2 Visual and Performing Arts Core: Applied ...,This is an intensive acting course that builds...
1255,PY,Psychology,PY 0435,Psychology of Personality,3 Credits,NaN,NaN,NaN,NaN,This course takes a comprehensive approach to ...
849,MOT,Management of Technology,MOT 0515,Independent Study,3 Credits,NaN,NaN,NaN,NaN,This course is intended to broaden the student...
736,MG,Management,MG 0101,Introduction to Management,3 Credits,Sophomore standing.,NaN,NaN,NaN,"This course integrates, through theory and its..."


In [39]:
df_catalog = pd.concat([dfcc17_18,dfcc18_19], axis=0)
df_catalog.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
405,AC,Accounting,AC 0590,Research on Contemporary Issues in Accounting,3 Credits,NaN,NaN,NaN,NaN,This course is a designated research course. I...
68,AS,American Studies,AS 0499,Independent Study,3 Credits,NaN,NaN,NaN,NaN,Students arrange for independent study with a ...
1359,MU,Music,MU 0200,Special Topics (Shell),3 Credits,One introductory or 100-level music class.,NaN,NaN,NaN,Students will undertake an in-depth study of a...
486,PO,Politics,PO 0112,21st Century Power Politics,3 Credits,NaN,NaN,NaN,PMPT Politics Major: Political Theory,How does power operate in the 21st century? Th...
508,SL,TESOL and Bilingual Education,SL 0582,TESOL/Bilingual Student Teaching and DSAP I Se...,3 Credits,NaN,NaN,NaN,NaN,Candidates take this weekly seminar concurrent...


In [40]:
df_catalog.to_sql('import_course_catalog', conn, if_exists='append', index=False)

In [41]:
%%sql
SELECT *
FROM import_course_catalog
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."
BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets."
BU,Business,BU 0220,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective","This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights."
BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments."
BU,Business,BU 0312,The Law of Business Organizations and Financial Transactions,3 Credits,BU 0211.,None,None,None,"This course offers an analysis of legal principles related to the law of agency, sole proprietorships, partnerships, corporations, limited liability companies, and other business forms. The second half of the course addresses several sections of the Uniform Commercial Code, such as negotiable instruments, bank collections and deposits and secured transactions. Finally, the course examines the law of suretyship, debtor-creditor relationships, and bankruptcy."
BU,Business,BU 0320,Employment Law and Discrimination in the Workplace,3 Credits,None,None,None,"MGEL Management: General Elective, UDIV U.S. Diversity","This course examines a variety of legal issues related to the workplace including the doctrine of employment at will, employee privacy, and the history and development of labor unions and the legal protections afforded by the National Labor Relations Act. A study of the

## Populate ERD Tables with data from Import Tables ##

In [42]:
%%sql

--Populating TIME table using import_course_meeting data-- 

INSERT INTO TIME (Start, End, Day)
SELECT DISTINCT start, end, day
FROM import_course_meeting

 * sqlite:///CourseData2.db
Done.


[]

In [43]:
%%sql 

SELECT * 
FROM TIME
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


TID,Start,End,Day
1,2014-09-02T08:00:00,2014-09-02T09:15:00,T
2,2014-09-05T08:00:00,2014-09-05T09:15:00,F
3,2014-09-09T08:00:00,2014-09-09T09:15:00,T
4,2014-09-12T08:00:00,2014-09-12T09:15:00,F
5,2014-09-16T08:00:00,2014-09-16T09:15:00,T


In [44]:
%%sql

--Populating Course_Meeting table using import_course_meeting data-- 

INSERT INTO Course_Meeting (CRN, Term, Location, TID)
SELECT DISTINCT crn, term, location, TID
FROM import_course_meeting
    JOIN TIME ON (import_course_meeting.Start = TIME.Start);

 * sqlite:///CourseData2.db
Done.


[]

In [45]:
%%sql 

--Checking data population of Course_Meeting table-- 

SELECT * 
FROM Course_Meeting
ORDER BY TID DESC
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


MID,CRN,Term,Location,TID
1156696,11802,Winter2018,DSB 115,42551
1156729,11833,Winter2018,DSB 109,42551
1156694,11802,Winter2018,DSB 115,42550
1156727,11833,Winter2018,DSB 109,42550
1156634,11803,Winter2018,DSB 114,42549


In [46]:
%%sql

--Populating PROGRAM table using import_course_catalog data-- 

INSERT INTO PROGRAM(PName, PCode)
SELECT DISTINCT program_name, program_code
FROM import_course_catalog;

 * sqlite:///CourseData2.db
Done.


[]

In [47]:
%%sql 

SELECT * 
FROM PROGRAM
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


PID,PName,PCode
1,Asian Studies,AN
2,Business,BU
3,Black Studies,BL
4,Bioengineering,BEN
5,Arabic,AR
6,Anthropology,AY
7,American Studies,AS
8,Japanese,JA
9,Art History,AH
10,Applied Ethics,AE


In [48]:
%%sql 

--Populating Catalog table using import_course_catalog data--  

INSERT INTO CATALOG (CatID, Title, Description, Credits, PID, PREREQS, COREQS, FEES, Attribute)
SELECT DISTINCT catalog_id, course_title, description, credits, PID, prereqs, coreqs, fees, attributes
FROM import_course_catalog
    JOIN PROGRAM ON (import_course_catalog.program_name = program.PName);

 * sqlite:///CourseData2.db
Done.


[]

In [49]:
%%sql 

--Checking data population of Catalog table-- 

SELECT * 
FROM CATALOG
LIMIT 2;

 * sqlite:///CourseData2.db
Done.


CatClass,CatID,Title,Description,Credits,PID,PREREQS,COREQS,FEES,Attribute
1,AN 0301,Independent Study,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,1-3 Credits,1,None,None,None,None
2,AN 0310,Asian Studies Seminar,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",3 Credits,1,None,None,None,None


In [50]:
%%sql 

--Populating Instructor table using import_courses data--  

INSERT INTO Instructor (Name)
SELECT DISTINCT primary_instructor
FROM import_course
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE 'primary_instructor' AND primary_instructor NOT LIKE '%/';

 * sqlite:///CourseData2.db
Done.


[]

In [51]:
%%sql 

--Checking data population of Instructor table-- 

SELECT * 
FROM Instructor
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


InID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [52]:
%%sql 

--Populating the SECTION table using import_course data--
--Joining Instructor table using import_course and instructor table--

INSERT INTO SECTION (CRN, Term, Section, Cap, Act, Rem, CatID, InID)
SELECT crn, term, section, cap, act, rem, catalog_id, InID
FROM import_course
    JOIN Instructor ON (import_course.primary_instructor = Instructor.Name)
    JOIN Catalog ON (import_course.catalog_id = Catalog.CatID)
;

 * sqlite:///CourseData2.db
Done.


[]

In [53]:
%%sql 

--Checking data population of Section table--

SELECT * 
FROM Section
LIMIT 30;

 * sqlite:///CourseData2.db
Done.


SID,CRN,Term,Section,Cap,Act,Rem,CatID,InID
1,70384,Fall2014,C01,0,31,-31,AC 0011,1
2,70385,Fall2014,C02,0,31,-31,AC 0011,1
3,70382,Fall2014,C03,0,31,-31,AC 0011,1
4,70291,Fall2014,C04,0,29,-29,AC 0011,2
5,70350,Fall2014,C05,0,30,-30,AC 0011,2
6,70381,Fall2014,C06,0,31,-31,AC 0011,2
7,70383,Fall2014,G,30,31,-1,AC 0011,3
8,70391,Fall2014,H,30,32,-2,AC 0011,4
9,71105,Fall2014,I,30,33,-3,AC 0011,4
10,71123,Fall2014,J,30,32,-2,AC 0011,4


## Drop Import Tables ##

In [54]:
%%sql
--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.


[]